In [2]:
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import RMSprop, Adadelta, Adam
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

dig = load_digits()
onehot_target = pd.get_dummies(dig.target)
x_train, x_val, y_train, y_val = train_test_split(dig.data, onehot_target, test_size=0.1, random_state=20)

model = Sequential()
model.add(Dense(128, input_dim=x_train.shape[1], activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(optimizer=Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(x_train, y_train, epochs=50, batch_size=64)

scores = model.evaluate(x_val, y_val)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 26,122
Trainable params: 26,122
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1617/1617 [==============================] - 0s 131us/step - loss: 2.0690 - categorical_accuracy: 0.4434
Epoch 2/50
1617/1617 [==============================] - 0s 26us/step - loss: 1.4361 - categorical_accuracy: 0.8281
Epoch 3/50
1617/1617 [==============================] - 0s 25us/step - loss: 0.9554 - categorical_accuracy: 0.8930
Epoch 4/50
1617/1617 [==============================] - 

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

dig = load_digits()
onehot_target = pd.get_dummies(dig.target)
x_train, x_val, y_train, y_val = train_test_split(dig.data, onehot_target, test_size=0.1, random_state=20)

def sigmoid(s):
    return 1/(1 + np.exp(-s))

def sigmoid_derv(s):
    return s * (1 - s)

def softmax(s):
    exps = np.exp(s - np.max(s, axis=1, keepdims=True))
    return exps/np.sum(exps, axis=1, keepdims=True)

def cross_entropy(pred, real):
    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples

def error(pred, real):
    n_samples = real.shape[0]
    logp = - np.log(pred[np.arange(n_samples), real.argmax(axis=1)])
    loss = np.sum(logp)/n_samples
    return loss

class MyNN:
    def __init__(self, x, y):
        self.x = x
        neurons = 128
        self.lr = 0.5
        ip_dim = x.shape[1]
        op_dim = y.shape[1]

        self.w1 = np.random.randn(ip_dim, neurons)
        self.b1 = np.zeros((1, neurons))
        self.w2 = np.random.randn(neurons, neurons)
        self.b2 = np.zeros((1, neurons))
        self.w3 = np.random.randn(neurons, op_dim)
        self.b3 = np.zeros((1, op_dim))
        self.y = y

    def feedforward(self):
        z1 = np.dot(self.x, self.w1) + self.b1
        self.a1 = sigmoid(z1)
        z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = sigmoid(z2)
        z3 = np.dot(self.a2, self.w3) + self.b3
        self.a3 = sigmoid(z3)
        
    def backprop(self):
        loss = error(self.a3, self.y)
        print('Error :', loss)
        a3_delta = cross_entropy(self.a3, self.y) # w3
        z2_delta = np.dot(a3_delta, self.w3.T)
        a2_delta = z2_delta * sigmoid_derv(self.a2) # w2
        z1_delta = np.dot(a2_delta, self.w2.T)
        a1_delta = z1_delta * sigmoid_derv(self.a1) # w1

        self.w3 -= self.lr * np.dot(self.a2.T, a3_delta)
        self.b3 -= self.lr * np.sum(a3_delta, axis=0, keepdims=True)
        self.w2 -= self.lr * np.dot(self.a1.T, a2_delta)
        self.b2 -= self.lr * np.sum(a2_delta, axis=0)
        self.w1 -= self.lr * np.dot(self.x.T, a1_delta)
        self.b1 -= self.lr * np.sum(a1_delta, axis=0)

    def predict(self, data):
        self.x = data
        self.feedforward()
        return self.a3.argmax()
			
model = MyNN(x_train/16.0, np.array(y_train))

epochs = 1500
for x in range(epochs):
    model.feedforward()
    model.backprop()
		
def get_acc(x, y):
    acc = 0
    for xx,yy in zip(x, y):
        s = model.predict(xx)
        if s == np.argmax(yy):
            acc +=1
    return acc/len(x)*100
	
print("Training accuracy : ", get_acc(x_train/16, np.array(y_train)))
print("Test accuracy : ", get_acc(x_val/16, np.array(y_val)))

Error : 3.701483199386804
Error : 11.652403058994116
Error : 10.25926385442499
Error : 8.911341734618691
Error : 6.3919959829477
Error : 4.80799555077005
Error : 3.98039848974918
Error : 2.93571651969515
Error : 1.7206189702740529
Error : 3.29613193278058
Error : 1.160482164993384
Error : 2.4973004043837626
Error : 0.991046259525647
Error : 1.805315817898198
Error : 1.0702942438461724
Error : 1.4397888537445387
Error : 1.0983278687032367
Error : 1.449969091507976
Error : 1.1204198225679625
Error : 1.0586387673775248
Error : 0.9933129883679168
Error : 1.0334050699983495
Error : 0.9012680858290718
Error : 1.1223596157815667
Error : 0.8622766457471867
Error : 0.9497117823495543
Error : 0.7457760553648534
Error : 0.9533199469835542
Error : 0.7310448873971991
Error : 0.8397131968721875
Error : 0.6762360791848827
Error : 0.8236584509932822
Error : 0.6637648550154206
Error : 0.7475987620277228
Error : 0.6408396701216902
Error : 0.7387204173052965
Error : 0.6470611579154623
Error : 0.679740804

Error : 0.12155809647800112
Error : 0.12118802240585809
Error : 0.12082002376210678
Error : 0.12045408251628559
Error : 0.12009018090939899
Error : 0.1197283014470426
Error : 0.11936842689275232
Error : 0.11901054026156722
Error : 0.11865462481379538
Error : 0.11830066404897498
Error : 0.11794864170001995
Error : 0.11759854172754389
Error : 0.11725034831435265
Error : 0.11690404586009899
Error : 0.1165596189760927
Error : 0.1162170524802593
Error : 0.1158763313922408
Error : 0.1155374409286337
Error : 0.11520036649835871
Error : 0.11486509369815696
Error : 0.11453160830820824
Error : 0.1141998962878673
Error : 0.11386994377151421
Error : 0.11354173706451498
Error : 0.11321526263928915
Error : 0.11289050713148145
Error : 0.11256745733623456
Error : 0.11224610020456044
Error : 0.11192642283980755
Error : 0.1116084124942221
Error : 0.11129205656560091
Error : 0.11097734259403433
Error : 0.11066425825873734
Error : 0.11035279137496663
Error : 0.11004292989102317
Error : 0.10973466188533801

Error : 0.061828976401079955
Error : 0.06171887228145339
Error : 0.06160909728947405
Error : 0.06149964991622663
Error : 0.061390528662818925
Error : 0.061281732040294315
Error : 0.061173258569545336
Error : 0.06106510678122782
Error : 0.060957275215676114
Error : 0.06084976242281879
Error : 0.06074256696209575
Error : 0.06063568740237549
Error : 0.06052912232187369
Error : 0.06042287030807246
Error : 0.06031692995764021
Error : 0.06021129987635273
Error : 0.06010597867901459
Error : 0.06000096498938179
Error : 0.05989625744008475
Error : 0.05979185467255259
Error : 0.05968775533693766
Error : 0.05958395809204129
Error : 0.05948046160523993
Error : 0.059377264552412407
Error : 0.05927436561786764
Error : 0.05917176349427327
Error : 0.05906945688258497
Error : 0.058967444491976544
Error : 0.058865725039770694
Error : 0.05876429725137045
Error : 0.05866315986019159
Error : 0.05856231160759547
Error : 0.05846175124282269
Error : 0.05836147752292742
Error : 0.058261489212712535
Error : 0.0

Error : 0.03908094690792146
Error : 0.03902879575807972
Error : 0.03897675660276562
Error : 0.038924829109971235
Error : 0.03887301294892735
Error : 0.03882130779009798
Error : 0.038769713305174494
Error : 0.03871822916707016
Error : 0.038666855049914545
Error : 0.038615590629048055
Error : 0.03856443558101647
Error : 0.03851338958356569
Error : 0.03846245231563636
Error : 0.03841162345735858
Error : 0.03836090269004675
Error : 0.0383102896961944
Error : 0.038259784159469096
Error : 0.038209385764707456
Error : 0.03815909419791001
Error : 0.03810890914623641
Error : 0.038058830298000446
Error : 0.03800885734266521
Error : 0.03795898997083821
Error : 0.03790922787426678
Error : 0.037859570745833224
Error : 0.03781001827955019
Error : 0.03776057017055606
Error : 0.03771122611511032
Error : 0.03766198581058903
Error : 0.03761284895548036
Error : 0.03756381524938006
Error : 0.0375148843929871
Error : 0.03746605608809922
Error : 0.037417330037608644
Error : 0.03736870594549773
Error : 0.037

Error : 0.027515088933899787
Error : 0.02748550241287684
Error : 0.027455968075732185
Error : 0.027426485805176506
Error : 0.027397055484272518
Error : 0.027367676996433065
Error : 0.027338350225419358
Error : 0.027309075055339027
Error : 0.027279851370644332
Error : 0.027250679056130413
Error : 0.027221557996933342
Error : 0.027192488078528466
Error : 0.027163469186728517
Error : 0.027134501207681874
Error : 0.027105584027870805
Error : 0.027076717534109694
Error : 0.027047901613543292
Error : 0.027019136153644974
Error : 0.026990421042215072
Error : 0.026961756167379024
Error : 0.026933141417585862
Error : 0.02690457668160632
Error : 0.026876061848531304
Error : 0.026847596807770137
Error : 0.0268191814490489
Error : 0.026790815662408773
Error : 0.026762499338204505
Error : 0.026734232367102633
Error : 0.026706014640079895
Error : 0.026677846048421675
Error : 0.026649726483720396
Error : 0.02662165583787386
Error : 0.0265936340030838
Error : 0.02656566087185411
Error : 0.026537736336

Error : 0.020802224697601086
Error : 0.020783783287752818
Error : 0.020765370255058025
Error : 0.020746985541095855
Error : 0.020728629087580632
Error : 0.020710300836361592
Error : 0.020692000729422587
Error : 0.02067372870888183
Error : 0.02065548471699165
Error : 0.020637268696138144
Error : 0.02061908058884102
Error : 0.020600920337753244
Error : 0.020582787885660793
Error : 0.020564683175482417
Error : 0.020546606150269327
Error : 0.02052855675320493
Error : 0.02051053492760459
Error : 0.020492540616915376
Error : 0.020474573764715728
Error : 0.020456634314715238
Error : 0.020438722210754368
Training accuracy :  100.0
Test accuracy :  97.77777777777777
